In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel(r'PQ-278 Split and Transpose.xlsx', usecols = 'A', header = None, nrows = 10)
df = df.iloc[:,0].str.split(' - ', expand = True)
df.columns = df.iloc[0]
df = df.iloc[1:]
df['Date'] = pd.to_datetime(df['Date'], format = '%m/%d/%y')
df['Month Start'] = df['Date'].dt.to_period('M').dt.to_timestamp()
df = df.astype({'Revenue':'float', 'Cost':'float'})
df['Profit'] = df['Revenue'] - df['Cost']
df

,Date,Org,Revenue,Cost,Month Start,Profit
1,2025-01-23,A,560.0,380.0,2025-01-01,180.0
2,2025-01-24,C,450.0,410.0,2025-01-01,40.0
3,2025-03-01,B,90.0,80.0,2025-03-01,10.0
4,2025-03-02,B,390.0,280.0,2025-03-01,110.0
5,2025-04-19,A,890.0,340.0,2025-04-01,550.0
6,2025-05-02,A,560.0,230.0,2025-05-01,330.0
7,2025-05-12,C,840.0,560.0,2025-05-01,280.0
8,2025-06-23,C,670.0,690.0,2025-06-01,-20.0
9,2025-06-25,B,880.0,470.0,2025-06-01,410.0


In [3]:
all_months = set(pd.Series(pd.date_range(df['Month Start'].min(), df['Month Start'].max(), freq = 'MS')))
available_months = set(df['Month Start'].unique())
df = pd.concat([df, pd.DataFrame(all_months - available_months, columns = ['Month Start'])], ignore_index = True)
df

,Date,Org,Revenue,Cost,Month Start,Profit
0,2025-01-23,A,560.0,380.0,2025-01-01,180.0
1,2025-01-24,C,450.0,410.0,2025-01-01,40.0
2,2025-03-01,B,90.0,80.0,2025-03-01,10.0
3,2025-03-02,B,390.0,280.0,2025-03-01,110.0
4,2025-04-19,A,890.0,340.0,2025-04-01,550.0
5,2025-05-02,A,560.0,230.0,2025-05-01,330.0
6,2025-05-12,C,840.0,560.0,2025-05-01,280.0
7,2025-06-23,C,670.0,690.0,2025-06-01,-20.0
8,2025-06-25,B,880.0,470.0,2025-06-01,410.0
9,NaT,NaN,NaN,NaN,2025-02-01,NaN


In [4]:
pivot = df.pivot_table(values = 'Profit', index = 'Org', columns = 'Month Start', aggfunc = 'sum', dropna = False)
pivot.columns = pivot.columns.month_name().str[:3]
pivot = pivot.drop(index = np.nan)
pivot.loc['Total'] = pivot.sum(numeric_only = True)
# pivot = pivot.reset_index()
# pivot.columns.name = None
# pivot.replace(np.nan, 0)
pivot

Month Start,Jan,Feb,Mar,Apr,May,Jun
Org,,,,,,
A,180.0,NaN,NaN,550.0,330.0,NaN
B,NaN,NaN,120.0,NaN,NaN,410.0
C,40.0,NaN,NaN,NaN,280.0,-20.0
Total,220.0,0.0,120.0,550.0,610.0,390.0
